<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/Exercicio5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5. Para a base de dados Auto MPG (disponibilizada em https://archive.ics.uci.edu/ml/datasets/Auto+MPG) faça:**

**a) Baixe o arquivo auto-mpg.data, remova as linhas que tem interrogação (?) e remova a última coluna (por quê?). Com as 150 primeiras linhas obtenha um modelo de regressão linear multivariada para predizer o valor da primeira variável (mpg). Avalie o resultado sobre o restante da base de dados, usando a métrica RMSE.**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option('mode.chained_assignment', None)
auto_mpg = pd.read_csv("auto-mpg.data", sep="\s+", header=None)

#Remoção da última coluna que possui informação textual irrelevante para a modelagem de dados
auto_mpg = auto_mpg.iloc[:,:8]

#Exclusão das linhas com interrogação
#print(auto_mpg.shape)
nan_value = float("NaN")
auto_mpg.replace('?', nan_value, inplace=True)
auto_mpg.dropna(inplace=True)
auto_mpg[3] = pd.to_numeric(auto_mpg[3])
#print(auto_mpg.info())

#Separação de dados para teste e para treino
mpg_train = auto_mpg.iloc[:150]
mpg_test = auto_mpg.iloc[150:]

#Prepara matriz para calcular coeficientes
x = mpg_train
t = mpg_train[0]
x[0] = 1

#Matriz de coeficientes w
w = np.dot(np.dot(pd.DataFrame(np.linalg.pinv(np.dot(x.T, x))), x.T), t)
print("Coeficientes:", w, "\n")

#Métrica RMSE
sum_err = 0
for i,row in mpg_test.iterrows():
  fx = w[0]
  for j in range(1, w.size):
    fx = fx + (row[j] * w[j])
    
  sum_err = sum_err + np.power(row[0]-fx, 2)

rmse = np.sqrt(sum_err / mpg_test.shape[0])
print("RMSE amostras de teste = {}".format(rmse))

Coeficientes: [ 3.04814971e+01 -2.18759449e-01  1.20959497e-03 -2.02132000e-02
 -3.89301557e-03 -2.28287159e-02  4.25123785e-02  1.37375313e+00] 

RMSE amostras de teste = 6.24700675673217


In [ ]:
##Código para teste
#import numpy as np
#from sklearn.linear_model import LinearRegression
#X = x
#y = t
#reg = LinearRegression().fit(X, y)
#print(reg.score(X, y))
#print(reg.coef_)
#print(reg.intercept_)
#reg.predict(np.array([[3, 5]]))

**b) Verifique quais são os atributos que estão relacionados com a saída: A partir dos coeficientes obtidos, aplique o teste F de Snedecor sobre cada variável individualmente (conforme nos slides). Indique quais foram os atributos que podem ser desconsiderados. Obtenha sobre o restante da base de dados a métrica RMSE com o modelo sem considerar esses atributos (não precisa estimar um novo modelo, só considere os valores dos coeficientes deles iguais a zero). Compare os resultados obtidos em a) e em b). Considere que os resíduos do modelo possui distribuição aproximadamente normal e que F[1,142] = 3,908.**

In [ ]:
#Teste F de Snedecor

#Cálculo do resíduo para modelo completo
rss = 0
for i,row in mpg_train.iterrows():
  fx = w[0]
  for j in range(1, w.size):
    fx = fx + (row[j] * w[j])
  
  rss = rss + np.power(row[0]-fx, 2)

#Calcula Teste F ao remover um atributo por vez
wf = np.array(np.zeros((w.size)))
for n in range(1,w.size):
  
  rss0 = 0
  np.copyto(wf, w)
  wf[n] = 0 #zera fator correspondente ao atributo a ser removido
  
  for i,row in mpg_train.iterrows():
    fx = wf[0]
    for j in range(1, wf.size):
      fx = fx + (row[j] * wf[j])
    
    rss0 = rss0 + np.power(row[0]-fx, 2)
  
  f = ((rss0 - rss)/(w.size - (w.size-2)))/ \
        (rss/(mpg_train.shape[0] - w.size - 1))

  msg = "pode"
  if(f > 3.908):
    msg = "não pode"
  
  print("Atributo {} {} ser desconsiderado".format(n, msg))

Atributo 1 não pode ser desconsiderado
Atributo 2 pode ser desconsiderado
Atributo 3 não pode ser desconsiderado
Atributo 4 não pode ser desconsiderado
Atributo 5 pode ser desconsiderado
Atributo 6 pode ser desconsiderado
Atributo 7 pode ser desconsiderado


In [ ]:
#Remoção de atributos que foram desconsiderados pelo teste F de Snedecor
np.copyto(wf, w)
wf[2] = 0
wf[5] = 0
wf[6] = 0
wf[7] = 0

#Métrica RMSE
sum_err = 0
for i,row in mpg_test.iterrows():
  fx = wf[0]
  for j in range(1, wf.size):
    fx = fx + (row[j-1] * wf[j])
    
  sum_err = sum_err + np.power(row[0]-fx, 2)

rmse = np.sqrt(sum_err / mpg_test.shape[0])
print("RMSE = {}".format(rmse))

RMSE = 9.462118886534151


Verifica-se que a remoção dos atributos 2, 5, 6 e 7 causaram o aumento do erro quadrático médio (de **6,247** para **9,462**). Portanto, mesmo o teste F de Sneider informando que as componentes podem ser desconsideradas, há uma piora no resultado da classificação.